# Ollama PDF RAG Notebook

### Import Libraries


In [1]:
# Imports
from langchain_community.document_loaders import UnstructuredPDFLoader
from langchain_ollama import OllamaEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import Chroma
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_ollama.chat_models import ChatOllama
from langchain_core.runnables import RunnablePassthrough
from langchain.retrievers.multi_query import MultiQueryRetriever

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')


## Load PDF

In [2]:
# Load PDF
local_path = "documents/scammer-agent.pdf"
if local_path:
    loader = UnstructuredPDFLoader(file_path=local_path)
    data = loader.load()
    print(f"PDF loaded successfully: {local_path}")
else:
    print("Upload a PDF file")

PDF loaded successfully: documents/scammer-agent.pdf


## Split text into chunks

In [3]:
# Split text into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = text_splitter.split_documents(data)
print(f"Text split into {len(chunks)} chunks")

Text split into 23 chunks


In [4]:
print(chunks[0])

page_content='4 2 0 2

t c O 1 2

] I

A . s c [

1 v 0 5 6 5 1 . 0 1 4 2 : v i X r a

Voice-Enabled AI Agents can Perform Common Scams

Richard Fang UIUC

Dylan Bowman UIUC

Daniel Kang UIUC

Abstract

Recent advances in multi-modal, highly ca- pable LLMs have enabled voice-enabled AI agents. These agents are enabling new applica- tions, such as voice-enabled autonomous cus- tomer service. However, with all AI capabili- ties, these new capabilities have the potential for dual use.

In this work, we show that voice-enabled AI agents can perform the actions necessary to perform common scams. To do so, we select a list of common scams collected by the govern- ment and construct voice-enabled agents with directions to perform these scams. We con- duct experiments on our voice-enabled agents and show that they can indeed perform the ac- tions necessary to autonomously perform such scams. Our results raise questions around the widespread deployment of voice-enabled AI agents.

1

Introducti

## Create vector database

In [5]:
# Create vector database
vector_db = Chroma.from_documents(
    documents=chunks,
    embedding=OllamaEmbeddings(model="nomic-embed-text"),
    collection_name="local-rag"
)
print("Vector database created successfully")

Vector database created successfully


In [6]:
vector_db.similarity_search(query=("Voice-Enabled AI Agents"), k=5)

[Document(metadata={'source': 'documents/scammer-agent.pdf'}, page_content='4 2 0 2\n\nt c O 1 2\n\n] I\n\nA . s c [\n\n1 v 0 5 6 5 1 . 0 1 4 2 : v i X r a\n\nVoice-Enabled AI Agents can Perform Common Scams\n\nRichard Fang UIUC\n\nDylan Bowman UIUC\n\nDaniel Kang UIUC\n\nAbstract\n\nRecent advances in multi-modal, highly ca- pable LLMs have enabled voice-enabled AI agents. These agents are enabling new applica- tions, such as voice-enabled autonomous cus- tomer service. However, with all AI capabili- ties, these new capabilities have the potential for dual use.\n\nIn this work, we show that voice-enabled AI agents can perform the actions necessary to perform common scams. To do so, we select a list of common scams collected by the govern- ment and construct voice-enabled agents with directions to perform these scams. We con- duct experiments on our voice-enabled agents and show that they can indeed perform the ac- tions necessary to autonomously perform such scams. Our results raise q

## Set up LLM and Retrieval

In [7]:
# Set up LLM and retrieval
local_model = "llama3.2"  # or whichever model you prefer
llm = ChatOllama(model=local_model)

In [8]:
# Query prompt template
QUERY_PROMPT = PromptTemplate(
    input_variables=["question"],
    template="""You are an AI language model assistant. Your task is to generate 2
    different versions of the given user question to retrieve relevant documents from
    a vector database. By generating multiple perspectives on the user question, your
    goal is to help the user overcome some of the limitations of the distance-based
    similarity search. Provide these alternative questions separated by newlines.
    Original question: {question}""",
)

# Set up retriever
retriever = MultiQueryRetriever.from_llm(
    vector_db.as_retriever(), 
    llm,
    prompt=QUERY_PROMPT
)

## Create chain

In [9]:
# RAG prompt template
template = """
Answer the question based on the context below. If you can't 
answer the question, reply "I don't know".

Context: {context}

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template)

In [10]:
# Create chain
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

## Chat with PDF

In [11]:

def chat_with_pdf(question, stream=False):
    """
    Chat with the PDF using the RAG chain.
    """
    if stream:
        response = ""
        for token in chain.stream({"question": question}):
            response += token
            print(token, end='', flush=True)
    else:
        print(chain.invoke({"question": question}))


In [12]:
# Example 1
chat_with_pdf("What is the main idea of this document?")

The main idea of this document appears to be the exploration of using AI technology for nefarious purposes, specifically in the context of scams and cybersecurity threats. The authors examine the capabilities of language models and propose an architecture for creating AI agents that can perform common scams, such as navigating websites and retrieving user credentials.


In [13]:
# Example 2
chat_with_pdf("What is the purpose of the scammer agent?", stream=True)

The purpose of the scammer agent is to conduct common scams in real-time, specifically phone-based scams. The agent uses voice-enabled AI capabilities to autonomously perform actions needed to conduct these scams, such as logging into bank accounts, completing two-factor authentication processes, and stealing funds from victims.

In [14]:
# Example 3
chat_with_pdf("Can you explain the case study highlighted in the document?", stream=True)

The case study highlighted in the document is a redacted transcript and an abridged action log for a bank transfer scam. The transcript shows the interaction between the scammer (posing as John from Bank of America) and the victim, where the scammer asks for the victim's username and password to verify their information for security purposes. After receiving the password, the scammer then requests the 2FA code and performs actions to fill out the form, navigate to the transfer page, search for a recipient, and transfer the money. The case study aims to understand the trajectory of these scams and highlight the capabilities of newly available technology in nefarious uses.

## Clean up (optional)

In [15]:
# Optional: Clean up when done 
vector_db.delete_collection()
print("Vector database deleted successfully")

Vector database deleted successfully
